In [1]:
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.linalg as sla
import pickle
import os
from sklearn.datasets import load_svmlight_file

from methods import GradientDescent, Newton, Nesterov, RBFGS, LBFGS, BFGS
from methods.rbfgs import Uniform, Gaussian, CustomDiscrete, ConstantDistribution, \
                          MatrixDistribution
from oracles import LogRegL2Oracle, create_log_reg_oracle, QuadraticOracle
from utils import select_basis_columns, run_rbfgs_experiment, run_all_methods

In [10]:
file = './datasets/covtype.libsvm.binary.scale'
A, b = load_svmlight_file(file, n_features=None)
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))

(n, d) = (581012, 54)
density = 0.22121166108381518


In [12]:
with open('./results/covtype/regcoef=1.00e-03/svd.pkl', 'rb') as file:
    U, sigma_diag, V = pickle.load(file)
print('Smoothness based on singular values: {}'.format(sigma_diag.max()**2 / (4. * A.shape[0])))
print('Smoothness based on row norms: {}'.format(np.max(npla.norm(A.toarray(), axis=1))**2 / 4.))

Smoothness based on singular values: 0.7855090943556806
Smoothness based on row norms: 1.9039613189055


In [15]:
sketch_sizes = [1, 5, 10, 20, 40, 50]
def experiment_covtype(regcoef):
    oracle = create_log_reg_oracle(A, b, regcoef)
    run_all_methods(
        oracle=oracle, 
        sketch_sizes=sketch_sizes, 
        max_iter=1000, 
        mat=A.toarray(), 
        output_folder='./results/covtype/regcoef={:.2e}'.format(regcoef), 
        sigma_tolerance=1e-8, 
        method_tolerance=1e-15, 
        stopping_criteria='grad_abs', 
        random_state=0, 
        methods=['gauss', 'coord', 'svd-no-sigma', 'bfgs', 'nesterov']
    )

In [16]:
%%time
experiment_covtype(1e-2)

Read SVD from ./results/covtype/regcoef=1.00e-02/svd.pkl
Singular values above tolerance: 53

RBFGS-SVD sketch no sigma... Done
RBFGS-gauss... Done
RBFGS-coord...Done
BFGS... Done

All runs completed.
CPU times: user 103 ms, sys: 399 ms, total: 502 ms
Wall time: 540 ms


In [17]:
%%time
experiment_covtype(1e-3)

Read SVD from ./results/covtype/regcoef=1.00e-03/svd.pkl
Singular values above tolerance: 53

RBFGS-SVD sketch no sigma... Done
RBFGS-gauss... Done
RBFGS-coord...Done
BFGS... Done

All runs completed.
CPU times: user 2h 11min 23s, sys: 3min 37s, total: 2h 15min 1s
Wall time: 5min 25s


### Nesterov

In [19]:
file = './datasets/covtype.libsvm.binary.scale'
A, b = load_svmlight_file(file, n_features=None)
regcoef = 1e-3
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))
oracle = create_log_reg_oracle(A, b, regcoef=regcoef)

(n, d) = (581012, 54)
density = 0.22121166108381518


In [22]:
np.random.seed(42)
x_0 = np.random.normal(size=A.shape[1])
output_folder = './results/covtype/regcoef={:.2e}'.format(regcoef)
method = Nesterov(oracle, x_0, stopping_criteria='grad_abs', 
                  tolerance=1e-16)
method.run(max_iter=50000, max_time=10)
method.oracle = None
with open('{}/nesterov.pkl'.format(output_folder), 'wb') as file:
    pickle.dump(method, file)